# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [39]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [40]:
# This checks current working directory
print(os.getcwd())

# This retrieves current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# This creates a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [41]:
full_data_rows_list = [] 
    
# This for-loop reads every filepath in the file path list 
for f in file_path_list:
# This allows reading each csv file  
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # For each csv file being read, this for-loops appends full_data_rows_list-list with the content of each file        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
#uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        # This skips an entire row if value of first column is an empty-string
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [42]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion. 

## Now we are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing Apache Cassandra code in the cells below

#### Creating a Cluster

In [43]:
# This makes a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# We create a session to establish connection and begin executing queries
session = cluster.connect()

#### Create Keyspace

In [44]:
# Create a Keyspace 
keyspace = '''CREATE KEYSPACE IF NOT EXISTS project2 
                WITH REPLICATION = {'class': 'SimpleStrategy', 'replication_factor': 1}'''

#### Set Keyspace

In [45]:
# Set KEYSPACE to the keyspace specified above
try:
    session.execute(keyspace)
except Exception as e:
    print(e)

try:
    session.set_keyspace('project2')
except Error as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra we model the database tables on the queries we want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [46]:
## Query 1:  This query provides the artist, song title and song's length in the music app history that was heard during \
## specifc sessionId (= 338) and itemInSession (= 4)

## Table name: session_info
## Column 1: artist_name
## Column 2: song_title
## Column 3: song_length
## PRIMARY Composite KEY: sessionId, itemInSession - In order to present specific data (according to sessionId and itemInSession)  

session_info = '''CREATE TABLE IF NOT EXISTS sessioninfo(
                  artist text,
                  song text,
                  song_length float,
                  session_Id int,
                  item_InSession int,
                  PRIMARY KEY ((session_Id, item_InSession))
                  )'''

# Here, we will execute the CREATE TABLE query using Cassandra's session  
try:
    session.execute(session_info)
except Error as e:
    print(e)
    
# Here, we read the condensed datafile created earlier (See codeblock # 3) row by row, and insert into specific attribute/column's values into table 'session_info' 
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    ## Assign placeholder in the INSERT statements & assign datafile's row-specifc data to the table's column by using Cassandra's session.execute()    
    ## Remember, indexing of columns from csv files starts from 0
    for line in csvreader:  
        query = "INSERT INTO sessioninfo(artist, song, song_length, session_Id, item_InSession) VALUES (%s,%s,%s,%s,%s)"
        session.execute(query, (line[0], line[9], float(line[5]), int(line[8]), int(line[3])))
        
## Here, we verify data from datafile has been added to table by using SELECT statement
selectfirst = '''SELECT artist, song, song_length FROM sessioninfo WHERE session_Id=338 AND item_InSession=4 '''
try:
    rows = session.execute(selectfirst)
except Error as e:
    print(e)

# Here we will print values from client's query-specific attributes 
for row in rows:
    print(type(row))
    print(row.artist, row.song, row.song_length)

<class 'cassandra.io.asyncorereactor.Row'>
Faithless Music Matters (Mark Knight Dub) 495.30731201171875


In [47]:
## Query 2: This query provides the artist, song title, and user's first and last name in the music app history \
## that was heard by specifc userId  (= 10) and of specific sessionId (= 182)

# Table name: user_session_table 
# Column 1: artist
# Column 2: song
# Column 3: first name
# Column 4: last name
# PRIMARY COMPOSITE KEY: (userid, sessionId)
# Clustering Column: itemInSession

user_session_table = '''CREATE TABLE IF NOT EXISTS user_session_table( 
                        artist text, 
                        song text, 
                        first_name text, 
                        last_name text, 
                        userId int, sessionId 
                        int, 
                        itemInSession int, 
                        PRIMARY KEY ((userId, sessionId), itemInSession)
                        )'''
                    
try:
    session.execute(user_session_table)
except Error as e:
    print(e)
                  
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
   
    for line in csvreader:
        query = "INSERT INTO user_session_table(artist, song, first_name, last_name, userId, sessionId, itemInSession) VALUES (%s,%s,%s,%s,%s,%s,%s)"
        session.execute(query, (line[0], line[9], line[1], line[4], int(line[10]), int(line[8]), int(line[3])))
        
## Here, we verify data from datafile has been added to table by using SELECT statement        
selectsecond = '''SELECT artist, song, first_name, last_name FROM user_session_table WHERE userId=10 AND sessionId=182 '''
try:
    rows = session.execute(selectsecond)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.song, row.first_name, row.last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [48]:
## Query 3: This query provides the user's first and last name in the music app history \
## who listened to song 'All Hands Against His Own' 

## Table name: name_table
# Column 1: first name
# Column 2: last name
# PRIMARY KEY: userid
# Clustering Column: sessionId

name_table = '''CREATE TABLE IF NOT EXISTS name_table
                (first_name text, last_name text, song text, userid int, PRIMARY KEY(song,userid))'''
try:
    session.execute(name_table)
except Error as e:
    print(e)

with open(file, encoding='utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = '''INSERT INTO name_table (first_name, last_name, song, userid) VALUES (%s,%s,%s,%s)'''
        session.execute(query, (line[1], line[4], line[9], int(line[10])))
        
selectthird = '''SELECT first_name, last_name FROM name_table WHERE song='All Hands Against His Own' '''

try:
    rows = session.execute(selectthird)
except Error as e:
    print(e)
    
for row in rows:
    print(row.first_name, row.last_name)
    
#session.execute('''DROP TABLE name_table''')

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [49]:
## TO-DO: Drop the table before closing out the sessions

session.execute('''DROP TABLE sessioninfo''')
session.execute('''DROP TABLE user_session_table''')
session.execute('''DROP TABLE name_table''')

# session.execute('''SELECT first_name, last_name FROM name_table WHERE song='All Hands Against His Own' ''')

### Close the session and cluster connection¶

In [50]:
session.shutdown()
cluster.shutdown()